# 课堂随记

## Books

### 3.SICP | 1.Artificial Intelligence A Modern Approach | 2.Programming Pearls | 4.Fluent Python | W

### Build a Modern Computer from First principles: from nerd to Tetris

# NIPS ACL ICML ICLR AAAI

1.Words -> 2.Entity -> 3.Syntax -> 4.Semantic -> 5.Dialogue -> 6.Reading Comprehension -> 7.Generation -> 8.Representation + Policy -> 1.Words

目前对话系统主要基于搜索，核心点是语义匹配

Reading Comprehension 也可以做自动摘要

Generation 可以做财务报表自动生成

AI ：  
1. Representation 表征  
2. policy 计算

why NLP？
* Information Chaos  
* Unstructured (单词和句子如何表征现在仍然解决不了，但作了很好的尝试:word2vec)
* Discrete（数据是不连续的）
* Unconventional(语言是不常规的，图像则不会）
* OOV（out of vocabulary)

---

## 数组压平实现

### itertools

In [118]:
import itertools
a = ["a", "b", ["c", ["d", "e"]]]
out = list(itertools.chain.from_iterable(a))
out

['a', 'b', 'c', ['d', 'e']]

### 函数递归实现

In [89]:
from icecream import ic
def flatten_list(l):
    ic(l)
    output = []
    for i in l:
        if type(i) == list:
            output+=flatten_list(i)  # 一般遇到list的情况下递归
        else:
            output.append(i)  #递归到最底层时的条件
    ic(output)
    return output

In [90]:
flatten_list([1, [2, 3,[4, 5]]])

ic| l: [1, [2, 3, [4, 5]]]
ic| l: [2, 3, [4, 5]]
ic| l: [4, 5]
ic| output: [4, 5]
ic| output: [2, 3, 4, 5]
ic| output: [1, 2, 3, 4, 5]


[1, 2, 3, 4, 5]

### lambda表达式递归实现

In [93]:
flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) in [tuple, list] else [x] 

In [94]:
flatten(a)

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [100]:
flat = lambda L: sum(map(flat, L), []) if isinstance(L, list) else [L]

In [101]:
flat(a)

[1, 2, 3, 4, 5, 6, (7, 8), 9]

---

## Dynamic Programming for cutting problem
* Rob Cutting Problem
* Edit-Distance Problem
* Key Characteristics for Dynamic Programming
* The Travel Salesman Problem

In [1]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

In [2]:
from collections import defaultdict

In [3]:
price = defaultdict(int)

In [4]:
for i, p in enumerate(original_price):
    price[i+1] = p

In [5]:
assert price[1] == 1

In [75]:
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
#         [r(n) + r(0)] + [r(i) + r(n-i) for i in range(1, n)]  这样递归会没有出口
    )

In [76]:
price[132]  # 说明肯定要拆分

0

In [19]:
r(8)

22

In [78]:
# !mv 图片路径 .  

In [10]:
solution = {}

In [80]:
from functools import lru_cache

In [81]:
@lru_cache(maxsize=2**10)
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key = lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    return max_price

In [82]:
solution = {}

In [84]:
r(100)

300

In [85]:
def fun_1(n):

    for i in range(n):
        print(n)


In [86]:
import time

In [87]:
def func_slow(n):
    start = time.time()
    for i in range(n):
        time.sleep(0.2)
        print(n)
    print('used time: {}'.format(time.time() -  start))

In [88]:
func_slow(5)

5
5
5
5
5
used time: 1.0232460498809814


In [89]:
def call_time(fun_1, arg): # 脚手架程序
    start = time.time()
    fun_1(arg)
    print('used time: {}'.format(time.time() -  start))

In [90]:
call_time(fun_1, 10)

10
10
10
10
10
10
10
10
10
10
used time: 0.0007200241088867188


In [91]:
def get_call_time(func):
    def _inner(arg):
        start = time.time()
        result = func(arg)
        print('used time: {}'.format(time.time() - start))
        return result
    return _inner

In [92]:
func_1_could_get_calltime  = get_call_time(fun_1)

In [93]:
func_1_could_get_calltime(10)

10
10
10
10
10
10
10
10
10
10
used time: 0.0011792182922363281


### 所以就可以用以下方法

In [94]:
fun_1 = get_call_time(fun_1)

In [95]:
fun_1(10)

10
10
10
10
10
10
10
10
10
10
used time: 0.0004620552062988281


### 为了避免复杂，所以可以用

In [102]:
@get_call_time
def fun_1(n):

    for i in range(n):
        print(n)

In [97]:
fun_1(10)

10
10
10
10
10
10
10
10
10
10
used time: 0.0017740726470947266


### @w a <=> a = w(a)

### 现在再继续改写

In [108]:
function_called_time = defaultdict(int)

def get_call_time(func):
    def _inner(arg):
        global function_called_time
        function_called_time[func.__name__] += 1
        start = time.time()
        result = func(arg)     # 关键点，都有返回一个result
        print('function call time is: {}'.format(function_called_time[func.__name__]))
        return result
    return _inner

In [109]:
fun_1(10)    # 可用于累计用户访问数

10
10
10
10
10
10
10
10
10
10
function call time is: 1


---

In [6]:
from icecream import ic

In [2]:
def memo(func):
    cache = {}
#     @wraps(func)
    def _wrap(n): ## ? *args, **kwargs
        if n in cache: result = cache[n]   # 都要返回原函数的result
        else:
            cache[n] = func(n)
            result = cache[n]
        return result
    return _wrap

In [7]:
cache2 = defaultdict(int)
def memo_test(func):
    def _inner(*args):
#         print(args[0])
        if args[0] in cache2: result = cache2[args[0]]
        else:
#             print(cache)
            cache2[args[0]] = func(args[0])
            result = cache2[args[0]]
        return result
    return _inner

In [8]:
@memo_test
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key = lambda x: x[0]
    )
    
    solution[n] = (split_point, n - split_point)
    
    return max_price

In [27]:
r(101)

301

In [22]:
solution[18]

(2, 16)

### Dynamic Programming 
不断查表的意思
+ 分析子问题的重复性
+ 子问题要进行存储
+ Solution要进行解析

In [19]:
def not_cut(spilt): return spilt == 0

def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    
    if not_cut(left): return [right]
    
    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)

In [28]:
parse_solution(101, solution)

[1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

### 左边等于0就停止，然后返回右边，[2] + parse_solution(16)

---

# Edit Distance
* Insertion
* Deletion
* Substitution

In [34]:
from functools import lru_cache

In [59]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        
    ]
    
    return min(
        edit_distance(string1[:-1], string2) + 1,
        edit_distance(string1, string2[:-1]) + 1,
        edit_distance(string1[:-1], string2[:-1]) + (2 if tail_s1 != tail_s2 else 0)
    )

In [60]:
edit_distance('我今天确实不太想吃饭', '我今天确实不太像吃饭')

2

In [61]:
from icecream import ic

In [35]:
th = 0
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    global th 
        
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        # 先运行第一步，string1先递归到最后，
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 2, 'SUB {} => {}'.format(tail_s1, tail_s2))
    
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    th += 1
    
    ic(string1, string2, min_distance, operation, th)
    
    solution[(string1, string2)] = operation
    
    return min_distance

In [36]:
edit_distance('100', '110')

ic| string1: '1', string2: '1', min_distance: 0, operation: '', th: 1
ic| string1: '1'
    string2: '11'
    min_distance: 1
    operation: 'ADD 1'
    th: 2
ic| string1: '1'
    string2: '110'
    min_distance: 2
    operation: 'ADD 0'
    th: 3
ic| string1: '10'
    string2: '1'
    min_distance: 1
    operation: 'DEL 0'
    th: 4
ic| string1: '10'
    string2: '11'
    min_distance: 2
    operation: 'DEL 0'
    th: 5
ic| string1: '10'
    string2: '110'
    min_distance: 1
    operation: ''
    th: 6
ic| string1: '100'
    string2: '1'
    min_distance: 2
    operation: 'DEL 0'
    th: 7
ic| string1: '100'
    string2: '11'
    min_distance: 3
    operation: 'DEL 0'
    th: 8
ic| string1: '100'
    string2: '110'
    min_distance: 2
    operation: 'DEL 0'
    th: 9


2

In [174]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73:

In [69]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73:

In [69]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73:

In [69]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73:

---

### 解决蚂蚁走路问题

In [41]:
input1 = '''
5
1
6 3
8 2 6
2 1 6 5
3 2 4 7 6
'''

In [74]:
import numpy as np
store = np.zeros((5,5), int)
store

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [75]:
nums = defaultdict(list)
for i, p in enumerate(input1.split('\n')):
    if not p: continue
    p = p.strip()
    if i == 1: size = p
    else:
        for j, k in enumerate(p.split()):
            ic(i, j, store)
            store[i-2][j] = k
# print(len(input1.split('\n')))

ic| i: 2
    j: 0
    store: array([[0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0]])
ic| i: 3
    j: 0
    store: array([[1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0]])
ic| i: 3
    j: 1
    store: array([[1, 0, 0, 0, 0],
                  [6, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0]])
ic| i: 4
    j: 0
    store: array([[1, 0, 0, 0, 0],
                  [6, 3, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0]])
ic| i: 4
    j: 1
    store: array([[1, 0, 0, 0, 0],
                  [6, 3, 0, 0, 0],
                  [8, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0]])
ic| i: 4
    j: 2
    store: a

In [76]:
store

array([[1, 0, 0, 0, 0],
       [6, 3, 0, 0, 0],
       [8, 2, 6, 0, 0],
       [2, 1, 6, 5, 0],
       [3, 2, 4, 7, 6]])

### 递归解法

In [132]:
times = 0
def find_max(a, b):
    global times
    if a == 4: return store[a][b]
    times += 1
    ic(times)
    return store[a][b] + max(find_max(a+1,b), find_max(a+1, b+1))

In [133]:
find_max(0, 0)   #此方法计算15次

ic| times: 1
ic| times: 2
ic| times: 3
ic| times: 4
ic| times: 5
ic| times: 6
ic| times: 7
ic| times: 8
ic| times: 9
ic| times: 10
ic| times: 11
ic| times: 12
ic| times: 13
ic| times: 14
ic| times: 15


23

### 记忆搜索优化算法

In [96]:
dp = defaultdict()
def memo_for_ant(func):
    def _wrap(*args):
        if args in dp: return dp[args]
        else:
            result = func(*args)
            dp[args] = result
        return result
    return _wrap

In [97]:
import time

In [170]:
times = 0
@memo_for_ant
def find_max(a, b):
    global times
    if a == 4: return store[a][b]
    times += 1
    ic(times)
    return store[a][b] + max(find_max(a+1,b), find_max(a+1, b+1))

In [171]:
find_max(0, 0) # c此方法只计算了10次

ic| times: 1
ic| times: 2
ic| times: 3
ic| times: 4
ic| times: 5
ic| times: 6
ic| times: 7
ic| times: 8
ic| times: 9
ic| times: 10


23

In [168]:
dp = {}

为什么会产生重复值: 因为比如算a[2][0]要算a[3][0]和a[3][1]，a[2][1]要算a[3][1]和a[3][2]这里就有重复的a[3][1]

---

### 深度优先搜索

In [172]:
sum_1 = 0
Max = 0
def dfs(a, b):
#     ic(a,b,sum_1)
    global sum_1
    global Max
    ic(a,b,sum_1)
    sum_1 += store[a][b]  # 去到就加
    if a==4: 
        Max = sum_1 if sum_1 > Max else Max
        return Max
    for i in range(0, 2):
        dfs(a+1, b+i)
        sum_1 -= store[a+1][b+i]

In [173]:
dfs(0, 0)

ic| a: 0, b: 0, sum_1: 0
ic| a: 1, b: 0, sum_1: 1
ic| a: 2, b: 0, sum_1: 7
ic| a: 3, b: 0, sum_1: 15
ic| a: 4, b: 0, sum_1: 17
ic| a: 4, b: 1, sum_1: 17
ic| a: 3, b: 1, sum_1: 15
ic| a: 4, b: 1, sum_1: 16
ic| a: 4, b: 2, sum_1: 16
ic| a: 2, b: 1, sum_1: 7
ic| a: 3, b: 1, sum_1: 9
ic| a: 4, b: 1, sum_1: 10
ic| a: 4, b: 2, sum_1: 10
ic| a: 3, b: 2, sum_1: 9
ic| a: 4, b: 2, sum_1: 15
ic| a: 4, b: 3, sum_1: 15
ic| a: 1, b: 1, sum_1: 1
ic| a: 2, b: 1, sum_1: 4
ic| a: 3, b: 1, sum_1: 6
ic| a: 4, b: 1, sum_1: 7
ic| a: 4, b: 2, sum_1: 7
ic| a: 3, b: 2, sum_1: 6
ic| a: 4, b: 2, sum_1: 12
ic| a: 4, b: 3, sum_1: 12
ic| a: 2, b: 2, sum_1: 4
ic| a: 3, b: 2, sum_1: 10
ic| a: 4, b: 2, sum_1: 16
ic| a: 4, b: 3, sum_1: 16
ic| a: 3, b: 3, sum_1: 10
ic| a: 4, b: 3, sum_1: 15
ic| a: 4, b: 4, sum_1: 15
